In [3]:
from crawlbase import CrawlingAPI
from bs4 import BeautifulSoup
import pandas as pd

def crawl(page_url, api_token):
 # Initialize the CrawlingAPI object with your token
 api = CrawlingAPI({'token': api_token})
 # Get the page content
 response = api.get(page_url)
 # Check if the request was successful
 if response['status_code'] == 200:
  # Extract data
  target_data = scrape_data(response)
  return target_data
 
 else:
   print(f"Error: {response}")

def scrape_data(response):
    try:
        soup = BeautifulSoup(response['body'], 'html.parser')
        # Extract the title of the Crunchbase page
        title = soup.find('h1', class_='profile-name').get_text(strip=True)
        # Extract the description of the Crunchbase page
        description = soup.find('span', class_='description').get_text(strip=True)
        # Extract the location of the Crunchbase page
        location = soup.select_one('.section-content-wrapper li.ng-star-inserted').text.strip() # use select_one for css selectors
        # Extract industries
        industries = soup.select_one('.mat-mdc-card.mdc-card .text_and_value li.ng-star-inserted:nth-of-type(1) field-formatter')
        industry_names = [div.text.strip() for div in industries.find_all(class_="chip-text")]
        industry_string = ', '.join(industry_names) # join the list of industries to a string
        # Extract the employees of the Crunchbase page
        employees = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(2)').text.strip()
        # Extract the Company URL of the Crunchbase page
        company_url = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(5) a[role="link"]')['href']
        # Extract the Rank of the Crunchbase page
        rank = soup.select_one('.section-content-wrapper li.ng-star-inserted:nth-of-type(6) span').text.strip()
        # Extract the Founded Date of the Crunchbase page
        founded = soup.select_one('.mat-mdc-card.mdc-card .text_and_value li:nth-of-type(3) field-formatter').text.strip()
        # Extract the Founders of the Crunchbase page
        founders = soup.select_one('.mat-mdc-card.mdc-card .text_and_value li:nth-of-type(4) field-formatter').text.strip()
        
        return {
            'title': title,
            'description': description,
            'location': location,
            'industries': industry_string,
            'employees': employees,
            'company_url': company_url,
            'rank': rank,
            'founded': founded,
            'founders': founders,
        }
    except Exception as e:
        print(f"An error occurred: {e}")
        return {}
   




In [4]:
# Specify the Crunchbase page URL to scrape
page_url = 'https://www.crunchbase.com/organization/magic-leap'
# Specify your Crawlbase token (Confidential)
api_token = 'seVwrD4oKny7QfJr9iC0dA'

In [5]:
test = crawl(page_url, api_token)

In [52]:
test2 = crawl("https://www.crunchbase.com/organization/space-exploration-technologies", api_token)


Error: {'headers': {}, 'body': '', 'status_code': 520}


In [122]:
import pandas as pd
test_df = pd.DataFrame(test, index=[0])
display(test_df)

,title,description,location,industries,employees,company_url,rank,founded,founders
0,Magic Leap,Magic Leap is a proprietary wearable technolog...,"Plantation, Florida, United States","Augmented Reality, Computer Vision, Video, Wea...",1001-5000,http://magicleap.com,"25,926",2010,"Brian Schowengerdt, Rony Abovitz"


In [125]:
test_df2 = pd.DataFrame(test2, index=[1])
display(test_df2)

,title,description,location,industries,employees,company_url,rank,founded,founders
1,SpaceX,SpaceX is an aviation and aerospace company th...,"Hawthorne, California, United States","Advanced Materials, Aerospace, Manufacturing, ...",1001-5000,https://www.spacex.com,"14,482","May 6, 2002",Elon Musk


In [3]:
import os

os.chdir('C:/Users/Wan Hok Ming/Documents/Applied Policy Project/Crunchbase/html for parsing')

In [8]:
# function for getting all the urls of the organizations that the investor invested in

from urllib.parse import urljoin

def get_invest_urls(txt):
    with open(txt, 'r', encoding='utf-8') as file:
        investment_html = file.read()
    soup = BeautifulSoup(investment_html, 'html.parser')
    base_url = "https://www.crunchbase.com"
    invest_urls = soup.find_all('table')[0]
    data = invest_urls.find_all('a', href=True)
    href_items = [url['href'] for url in data]
    joined_href = [urljoin(base_url, url) for url in href_items]
    filtered_urls = [url for url in joined_href if url.startswith('https://www.crunchbase.com/organization/')]
    return(filtered_urls)

In [9]:
def get_tables(txt):
    with open(txt, 'r', encoding='utf-8') as file:
        investment_html = file.read()    
    soup = BeautifulSoup(investment_html, 'html.parser')
    invest_title = soup.find('thead')
    invest_title = invest_title.find_all('th')
    invest_title = [title.text.strip() for title in invest_title]
    invest_df = pd.DataFrame(columns=invest_title)
    invest_data = soup.find('tbody')
    invest_data = invest_data.find_all('tr')
    for data in invest_data:
        target = data.find_all('td')
        each_target = [obs.text.strip() if obs.text.strip() != "—" else None for obs in target]
        length = len(invest_df)
        invest_df.loc[length] = each_target
      
    new_columns = ['Location', 'Industries']
    for col in new_columns:
        invest_df[col] = ''
      
    return(invest_df)



In [50]:
test2 = crawl("https://www.crunchbase.com/organization/magic-leap", api_token)
test_df2 = pd.DataFrame(test2, index=[1])
display(test_df2)

Error: {'headers': {}, 'body': '', 'status_code': 520}


""
1


In [7]:
# extract tables from crunchbase html elements and export as csv

directory = "C:/Users/Wan Hok Ming/Documents/Applied Policy Project/Crunchbase/Invest"
directory2 = "C:/Users/Wan Hok Ming/Documents/Applied Policy Project/Crunchbase/invest_csv"

for txt in os.listdir(directory):
    if txt.endswith(".txt"):
        csv_convert = get_tables(os.path.join(directory, txt))
        csv_filename = os.path.splitext(txt)[0] + ".csv"
        csv_convert.to_csv(os.path.join(directory2, csv_filename), index=False)


In [18]:
import os
os.chdir("C:/Users/Wan Hok Ming/Documents/Applied Policy Project/Crunchbase/invest_csv/For me")

In [19]:
import pandas as pd
abc = pd.read_excel("KAUST Innovation Fund.xlsx")
abc

,Announced Date,Organization Name,Lead Investor,Funding Round,Money Raised,Location,Industries
0,2024-03-21,InCurA,Yes,Pre Seed Round - InCurA,NaN,"Cairo, Al Qahirah, Egypt",Manufacturing\nMedical\nMedical Device
1,2023-12-05,ClimateCrete,NaN,Series A - ClimateCrete,$3.3M,"San Jose, California, United States",Construction
2,2023-09-11,Barakah,NaN,Seed Round - Barakah,$1.5M,"Jeddah, Makkah, Saudi Arabia",Food and Beverage\nFood Delivery\nMarketplace
3,2023-09-08,SARsatX,No,Pre Seed Round - SARsatX,$800K,"Thuwal, Ha'il, Saudi Arabia",Analytics\nRemote Sensing
4,2023-02-01,AgShift,NaN,Venture Round - AgShift,NaN,"San Jose, California, United States",AgTech\nArtificial Intelligence (AI)\nComputer...
...,...,...,...,...,...,...,...
86,2010-10-19,Agendat,NaN,Pre Seed Round - Agendat,$49K,NaN,Apps\nScheduling
87,2010-10-01,Acadox,NaN,Seed Round - Acadox,$742.5K,"Riyadh, Ar Riyad, Saudi Arabia",E-Learning\nSoftware
88,2010-10-01,Red Sea Biogeographic Information System,NaN,Seed Round - Red Sea Biogeographic Information...,$56K,NaN,Information and Communications Technology (ICT)
89,2010-07-15,WebGLot,NaN,Seed Round - WebGLot,$169K,NaN,Information and Communications Technology (ICT)
